<a href="https://colab.research.google.com/github/AdibahShuib/MAT668/blob/main/AS_GUROBI1_VRPTW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gurobipy==11.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 7.7 MB/s eta 0:00:00


In [ ]:
import gurobipy as gp
from gurobipy import GRB

# INSTANCE CONFIGURATION
numCustomers = 9  # 9 customers plus the depot
maxNumVehicles = 6  # Increased number of vehicles
vehicleCapacity = 40  # Increased vehicle capacity
timeHorizon = 900  # Extended time horizon to 3:00 pm (900 minutes)

# Time matrix (depot + 9 customers)
time_matrix = [
    [0.00, 17.00, 60.80, 38.08, 23.88, 31.58, 50.05, 50.17, 50.17, 28.20],
    [16.82, 0.00, 55.80, 35.98, 21.78, 30.42, 48.83, 48.05, 48.05, 29.15],
    [60.65, 55.40, 0.00, 42.52, 47.17, 36.97, 19.80, 19.78, 19.78, 36.50],
    [36.30, 36.67, 43.40, 0.00, 19.92, 9.67, 27.57, 27.68, 27.68, 9.27],
    [21.35, 21.73, 46.75, 18.95, 0.00, 13.40, 32.10, 31.03, 31.03, 11.83],
    [30.13, 31.22, 37.95, 9.35, 14.47, 0.00, 22.12, 22.23, 22.23, 4.08],
    [48.47, 48.83, 19.80, 27.45, 32.10, 21.88, 0.00, 0.97, 0.97, 21.43],
    [48.73, 49.10, 19.75, 27.72, 32.35, 22.15, 1.08, 0.00, 0.00, 23.08],
    [48.73, 49.10, 19.75, 27.72, 32.35, 22.15, 1.08, 0.00, 0.00, 23.08],
    [28.20, 28.58, 38.80, 11.00, 11.83, 4.08, 22.97, 23.08, 23.08, 0.00]
]

# Convert the time matrix to a dictionary format suitable for Gurobi
travelTimes = {(i, j): time_matrix[i][j] for i in range(numCustomers + 1) for j in range(numCustomers + 1)}

# Reduced Service time to 2 minutes at each stop
serviceTimes = {i: 2 for i in range(numCustomers + 1)}

# More flexible uniform time window for all customers
timeWindows = {i: (600, timeHorizon) for i in range(numCustomers + 1)}

# Create model
model = gp.Model("VRPTW")

locations = [i for i in range(numCustomers + 1)]
connections = [(i, j) for i in locations for j in locations if i != j]

# Decision variable for routing
x = model.addVars(connections, vtype=GRB.BINARY, name="x")
model.setObjective(gp.quicksum(travelTimes[i, j] * x[i, j] for i, j in connections), GRB.MINIMIZE)

# Constraints
model.addConstrs((x.sum("*", j) == 1 for j in locations if j != 0), name="incoming")
model.addConstrs((x.sum(i, "*") == 1 for i in locations if i != 0), name="outgoing")
model.addConstr(x.sum(0, "*") <= maxNumVehicles, name="maxNumVehicles")

# Time Constraints - Adjusted for more flexibility
y = model.addVars(locations, name="y")
for i in locations:
    y[i].LB = timeWindows[i][0]
    y[i].UB = timeWindows[i][1]

model.addConstrs((y[i] + serviceTimes[i] + travelTimes[i, j] <= y[j] + (timeHorizon - y[j]) * (1 - x[i, j])
                  for i, j in connections if i != j), name="timeBigM")

# Solve the model
model.params.Threads = 4
model.optimize()

# Check for infeasibility
if model.status == GRB.INFEASIBLE:
    print('Model is infeasible')
    model.computeIIS()
    model.write("model.ilp")
else:
    # Solution Retrieval
    usedConnections = [(i, j) for (i, j) in x.keys() if x[i, j].X > 0.5]
    nextCustomer = {}
    for (i, j) in usedConnections:
        if i == 0:
            if 0 not in nextCustomer:
                nextCustomer[0] = []
            nextCustomer[0].append(j)
        else:
            nextCustomer[i] = j

    print(f"Solution contains {len(nextCustomer[0])} routes:")
    routeNumber = 0
    visitedCustomers = [False] * (numCustomers + 1)
    for firstCustomer in nextCustomer[0]:
        print(f"Route {routeNumber}: 0 -> ", end="")
        vehicleLoad = 0
        time = travelTimes[0, firstCustomer]
        violatedTimeWindows = False
        currentCustomer = firstCustomer
        while currentCustomer != 0:
            print(f"{currentCustomer} (L:{vehicleLoad}, T:{time}) -> ", end="")
            visitedCustomers[currentCustomer] = True
            vehicleLoad += 1  # Assuming each customer has a demand of 1 unit
            time = max(time, timeWindows[currentCustomer][0])
            if time > timeWindows[currentCustomer][1]:
                violatedTimeWindows = True
            time += (serviceTimes[currentCustomer] + travelTimes[currentCustomer, nextCustomer[currentCustomer]])
            currentCustomer = nextCustomer[currentCustomer]
        print(f"0 (L:{vehicleLoad}/{vehicleCapacity}, T:{time})")
        if vehicleLoad > vehicleCapacity:
            print("Vehicle capacity is exceeded!")
        if violatedTimeWindows:
            print("Time windows are violated!")
        routeNumber += 1

    print("Unvisited customers: ", end="")
    for c in range(1, numCustomers + 1):
        if not visitedCustomers[c]:
            print(f"{c}, ", end="")

    # Free resources
    model.dispose()
    gp.disposeDefaultEnv()

Restricted license - for non-production use only - expires 2025-11-24
Set parameter Threads to value 4
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 4 threads

         Reduce the value of the Threads parameter to improve performance


Optimize a model with 19 rows, 100 columns and 171 nonzeros
Model fingerprint: 0xeb2a59ba
Model has 90 quadratic constraints
Variable types: 10 continuous, 90 integer (90 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 9e+02]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 9e+02]
  RHS range        [1e+00, 6e+00]
  QRHS range       [8e+02, 9e+02]
Presolve added 90 rows and 0 columns
Presolve time: 0.01s
Presolved: 109 rows, 100 columns, 825 nonzeros
Variable types: 10 continuous, 90 intege